# Capstone Project  Week 5

## This code retrieves the Indian Restaurants and Art/History Museums near to The Marrakech Hotel, New York. It then generates a clustered map based on radial distances of the above vis a vis the Marrakech Hotel. 

## IMPORTING LIBRARIES AND INSTALLING FOLIUM

In [1]:
import pandas as pd
import numpy as np
import requests
import json

!conda install -c conda-forge geopy
from geopy.geocoders import Nominatim   #  converts addresses into latitude and longitude
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0
import folium

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML    

print('Folium installed')
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.26 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  11.21 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  10.40 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  32.85 MB/s
vincent-0.4.4- 100% |###################

## DEFINE FOURSQUARE CREDENTIALS

In [2]:
CLIENT_ID = 'OGSB4M544YINE2ZOU4U5K3BPJ1B3HXDCWTWQRDSJYMTHDPQB' # your Foursquare ID
CLIENT_SECRET = 'BPUSNB0OMFZJ4WDU5UW4XGAM0YCJZB0V54KTNPFUQ3TSP1LG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: OGSB4M544YINE2ZOU4U5K3BPJ1B3HXDCWTWQRDSJYMTHDPQB
CLIENT_SECRET:BPUSNB0OMFZJ4WDU5UW4XGAM0YCJZB0V54KTNPFUQ3TSP1LG


## USING GEOPY LIBRARY To Get Latitude And Longitude For Marakech Hotel

In [3]:
#   get latitude and longitude for Marrakech hotel, 2688 Broadway, NY

address = '2688 Broadway, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
ny_latitude = location.latitude
ny_longitude = location.longitude

print('The geograpical coordinate of Marakech Hotel are {}, {}.'.format(ny_latitude, ny_longitude))

The geograpical coordinate of Marakech Hotel are 40.79874195, -73.9682736223354.


## Do A search Query For "Indian Retaurants" at about a radius of 3000 m from The Marakech Hotel

In [4]:
search_query = 'Indian Restaurant'
LIMIT = 100
radius = 3000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ny_latitude,ny_longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=OGSB4M544YINE2ZOU4U5K3BPJ1B3HXDCWTWQRDSJYMTHDPQB&client_secret=BPUSNB0OMFZJ4WDU5UW4XGAM0YCJZB0V54KTNPFUQ3TSP1LG&ll=40.79874195,-73.9682736223354&v=20180605&query=Indian Restaurant&radius=3000&limit=100'

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cbfeb976a607121217d6ea9'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'primary': True,
      'shortName': 'Indian'}],
    'delivery': {'id': '319303',
     'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
       'prefix': 'https://fastly.4sqi.net/img/general/cap/',
       'sizes': [40, 50]},
      'name': 'seamless'},
     'url': 'https://www.seamless.com/menu/polash-indian-restaurant-2179-3rd-ave-new-york/319303?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=319303'},
    'hasPerk': False,
    'id': '4a60aa67f964a520f9c01fe3',
    'location': {'address': '2179 3rd Ave',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
   

###  Cleaning Of Json File And Insert Necessary Data Into Pandas Dataframe ny_dataframe_filtered

In [6]:
# Now we are ready to clean the json and structure it into a pandas dataframe.

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
ny_dataframe = json_normalize(venues)
ny_dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,319303,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/polash-indian-re...,False,4a60aa67f964a520f9c01fe3,2179 3rd Ave,...,"[2179 3rd Ave (at 119th St.), New York, NY 100...","[{'lat': 40.79974477464853, 'lng': -73.9387466...",40.799745,-73.938747,NaN,10035,NY,Polash Indian restaurant,v-1556081559,NaN
1,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,308759,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/clove-1592-amste...,False,54c2bd96498eaf5142e3fe92,1592 Amsterdam Ave,...,[1592 Amsterdam Ave (btwn 139th St & 138th St)...,"[{'lat': 40.82128, 'lng': -73.95062, 'label': ...",40.821280,-73.950620,NaN,10031,NY,Clove Indian Restaurant & Bar,v-1556081559,122224155
2,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,805476,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/ajanta-1237-amst...,False,41575800f964a5202e1d1fe3,1237 Amsterdam Ave,...,"[1237 Amsterdam Ave, New York, NY 10027, Unite...","[{'lat': 40.80990543935106, 'lng': -73.9588238...",40.809905,-73.958824,NaN,10027,NY,Ajanta Indian Restaurant,v-1556081559,NaN
3,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,296705,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/spice-hut-2172-2...,False,545eb9de498ed803ac5569c4,2036 2nd Ave,...,"[2036 2nd Ave (105 Street), New York, NY 10029...","[{'lat': 40.794197782323884, 'lng': -73.939836...",40.794198,-73.939837,NaN,10029,NY,SPICEHUT INDIAN RESTAURANT,v-1556081559,374556954
4,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,NaN,NaN,NaN,NaN,NaN,NaN,False,4e4e4d2fbd4101d0d7a73475,429 Malcolm X Blvd,...,"[429 Malcolm X Blvd, New York, NY 10037, Unite...","[{'lat': 40.8178800173299, 'lng': -73.94192061...",40.817880,-73.941921,NaN,10037,NY,Strength West Indian Restaurant,v-1556081559,NaN


In [7]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in ny_dataframe.columns if col.startswith('location.')] + ['id']
ny_dataframe_filtered = ny_dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
ny_dataframe_filtered['categories'] = ny_dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
ny_dataframe_filtered.columns = [column.split('.')[-1] for column in ny_dataframe_filtered.columns]

ny_dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Polash Indian restaurant,Indian Restaurant,2179 3rd Ave,US,New York,United States,at 119th St.,2490,"[2179 3rd Ave (at 119th St.), New York, NY 100...","[{'lat': 40.79974477464853, 'lng': -73.9387466...",40.799745,-73.938747,NaN,10035,NY,4a60aa67f964a520f9c01fe3
1,Clove Indian Restaurant & Bar,Indian Restaurant,1592 Amsterdam Ave,US,New York,United States,btwn 139th St & 138th St,2916,[1592 Amsterdam Ave (btwn 139th St & 138th St)...,"[{'lat': 40.82128, 'lng': -73.95062, 'label': ...",40.821280,-73.950620,NaN,10031,NY,54c2bd96498eaf5142e3fe92
2,Ajanta Indian Restaurant,Indian Restaurant,1237 Amsterdam Ave,US,New York,United States,NaN,1475,"[1237 Amsterdam Ave, New York, NY 10027, Unite...","[{'lat': 40.80990543935106, 'lng': -73.9588238...",40.809905,-73.958824,NaN,10027,NY,41575800f964a5202e1d1fe3
3,SPICEHUT INDIAN RESTAURANT,Indian Restaurant,2036 2nd Ave,US,New York,United States,105 Street,2449,"[2036 2nd Ave (105 Street), New York, NY 10029...","[{'lat': 40.794197782323884, 'lng': -73.939836...",40.794198,-73.939837,NaN,10029,NY,545eb9de498ed803ac5569c4
4,Strength West Indian Restaurant,Indian Restaurant,429 Malcolm X Blvd,US,New York,United States,NaN,3077,"[429 Malcolm X Blvd, New York, NY 10037, Unite...","[{'lat': 40.8178800173299, 'lng': -73.94192061...",40.817880,-73.941921,NaN,10037,NY,4e4e4d2fbd4101d0d7a73475
5,Tom's Restaurant,Diner,2880 Broadway,US,New York,United States,at W 112th St,785,"[2880 Broadway (at W 112th St), New York, NY 1...","[{'lat': 40.805436, 'lng': -73.965334, 'label'...",40.805436,-73.965334,NaN,10025,NY,415c9e00f964a520501d1fe3
6,Carmine's Italian Restaurant,Italian Restaurant,2450 Broadway,US,New York,United States,btwn W 90th & W 91st,978,"[2450 Broadway (btwn W 90th & W 91st), New Yor...","[{'lat': 40.7910963, 'lng': -73.9739914, 'labe...",40.791096,-73.973991,NaN,10024,NY,4a7778a1f964a5209be41fe3
7,Mexican Festival Restaurant,Mexican Restaurant,2672 Broadway,US,New York,United States,btwn 101st St & 102nd St,74,"[2672 Broadway (btwn 101st St & 102nd St), New...","[{'lat': 40.798280474035266, 'lng': -73.968912...",40.798280,-73.968913,NaN,10025,NY,525b396011d2467d9819abd9
8,Gabriela's Restaurant & Tequila Bar,Mexican Restaurant,688 Columbus Ave,US,New York,United States,at 93rd St.,836,"[688 Columbus Ave (at 93rd St.), New York, NY ...","[{'lat': 40.79123991711048, 'lng': -73.9687352...",40.791240,-73.968735,NaN,10025,NY,49f3ab02f964a520d16a1fe3
9,Fred's Restaurant,American Restaurant,476 Amsterdam Ave.,US,New York,United States,at W 83rd St,1614,"[476 Amsterdam Ave. (at W 83rd St), New York, ...","[{'lat': 40.785657789223166, 'lng': -73.976539...",40.785658,-73.976539,NaN,10024,NY,44281118f964a520ba311fe3


In [8]:
# remove rows where address is Nan because address is important for tourist

ny_dataframe_filtered = ny_dataframe_filtered.loc[ny_dataframe_filtered['categories'] == 'Indian Restaurant']
ny_dataframe_filtered = ny_dataframe_filtered.dropna(subset=['address'])
ny_dataframe_filtered = ny_dataframe_filtered.reset_index()
del ny_dataframe_filtered['index']
ny_dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Polash Indian restaurant,Indian Restaurant,2179 3rd Ave,US,New York,United States,at 119th St.,2490,"[2179 3rd Ave (at 119th St.), New York, NY 100...","[{'lat': 40.79974477464853, 'lng': -73.9387466...",40.799745,-73.938747,NaN,10035,NY,4a60aa67f964a520f9c01fe3
1,Clove Indian Restaurant & Bar,Indian Restaurant,1592 Amsterdam Ave,US,New York,United States,btwn 139th St & 138th St,2916,[1592 Amsterdam Ave (btwn 139th St & 138th St)...,"[{'lat': 40.82128, 'lng': -73.95062, 'label': ...",40.821280,-73.950620,NaN,10031,NY,54c2bd96498eaf5142e3fe92
2,Ajanta Indian Restaurant,Indian Restaurant,1237 Amsterdam Ave,US,New York,United States,NaN,1475,"[1237 Amsterdam Ave, New York, NY 10027, Unite...","[{'lat': 40.80990543935106, 'lng': -73.9588238...",40.809905,-73.958824,NaN,10027,NY,41575800f964a5202e1d1fe3
3,SPICEHUT INDIAN RESTAURANT,Indian Restaurant,2036 2nd Ave,US,New York,United States,105 Street,2449,"[2036 2nd Ave (105 Street), New York, NY 10029...","[{'lat': 40.794197782323884, 'lng': -73.939836...",40.794198,-73.939837,NaN,10029,NY,545eb9de498ed803ac5569c4
4,Strength West Indian Restaurant,Indian Restaurant,429 Malcolm X Blvd,US,New York,United States,NaN,3077,"[429 Malcolm X Blvd, New York, NY 10037, Unite...","[{'lat': 40.8178800173299, 'lng': -73.94192061...",40.817880,-73.941921,NaN,10037,NY,4e4e4d2fbd4101d0d7a73475
5,Curry & Kebab,Indian Restaurant,931 Amsterdam Ave,US,New York,United States,W 106th St.,263,"[931 Amsterdam Ave (W 106th St.), New York, NY...","[{'lat': 40.8002471075292, 'lng': -73.96585761...",40.800247,-73.965858,NaN,10025,NY,4aa573aaf964a5206d4820e3
6,Indian Tanpura,Indian Restaurant,564 Amsterdam Ave,US,New York,United States,87th st,1250,"[564 Amsterdam Ave (87th st), New York, NY 100...","[{'lat': 40.788514795983616, 'lng': -73.974404...",40.788515,-73.974405,NaN,10024,NY,4adb9df6f964a5206d2921e3
7,Alachi Masala,Indian Restaurant,488 Amsterdam Ave,US,New York,United States,NaN,1559,"[488 Amsterdam Ave, New York, NY 10024, United...","[{'lat': 40.7861109936332, 'lng': -73.97626996...",40.786111,-73.976270,NaN,10024,NY,4b0dec08f964a520ae5223e3
8,Swagat,Indian Restaurant,411 Amsterdam Ave,US,New York,United States,btwn 80th St & 79th St,1878,"[411 Amsterdam Ave (btwn 80th St & 79th St), N...","[{'lat': 40.783573082941615, 'lng': -73.978030...",40.783573,-73.978030,NaN,10024,NY,42489a80f964a5208b201fe3
9,Indigo Indian Bistro,Indian Restaurant,283 Columbus Ave,US,New York,United States,at 73rd St,2428,"[283 Columbus Ave (at 73rd St), New York, NY 1...","[{'lat': 40.778261683492374, 'lng': -73.978192...",40.778262,-73.978193,NaN,10023,NY,4a99ba54f964a520113020e3


In [9]:
# remove unnecessary columns
ny_dataframe_filtered.drop(ny_dataframe_filtered.columns[[3,4,5,6,9,12,13,14]], axis=1, inplace=True)
ny_dataframe_filtered

,name,categories,address,distance,formattedAddress,lat,lng,id
0,Polash Indian restaurant,Indian Restaurant,2179 3rd Ave,2490,"[2179 3rd Ave (at 119th St.), New York, NY 100...",40.799745,-73.938747,4a60aa67f964a520f9c01fe3
1,Clove Indian Restaurant & Bar,Indian Restaurant,1592 Amsterdam Ave,2916,[1592 Amsterdam Ave (btwn 139th St & 138th St)...,40.821280,-73.950620,54c2bd96498eaf5142e3fe92
2,Ajanta Indian Restaurant,Indian Restaurant,1237 Amsterdam Ave,1475,"[1237 Amsterdam Ave, New York, NY 10027, Unite...",40.809905,-73.958824,41575800f964a5202e1d1fe3
3,SPICEHUT INDIAN RESTAURANT,Indian Restaurant,2036 2nd Ave,2449,"[2036 2nd Ave (105 Street), New York, NY 10029...",40.794198,-73.939837,545eb9de498ed803ac5569c4
4,Strength West Indian Restaurant,Indian Restaurant,429 Malcolm X Blvd,3077,"[429 Malcolm X Blvd, New York, NY 10037, Unite...",40.817880,-73.941921,4e4e4d2fbd4101d0d7a73475
5,Curry & Kebab,Indian Restaurant,931 Amsterdam Ave,263,"[931 Amsterdam Ave (W 106th St.), New York, NY...",40.800247,-73.965858,4aa573aaf964a5206d4820e3
6,Indian Tanpura,Indian Restaurant,564 Amsterdam Ave,1250,"[564 Amsterdam Ave (87th st), New York, NY 100...",40.788515,-73.974405,4adb9df6f964a5206d2921e3
7,Alachi Masala,Indian Restaurant,488 Amsterdam Ave,1559,"[488 Amsterdam Ave, New York, NY 10024, United...",40.786111,-73.976270,4b0dec08f964a520ae5223e3
8,Swagat,Indian Restaurant,411 Amsterdam Ave,1878,"[411 Amsterdam Ave (btwn 80th St & 79th St), N...",40.783573,-73.978030,42489a80f964a5208b201fe3
9,Indigo Indian Bistro,Indian Restaurant,283 Columbus Ave,2428,"[283 Columbus Ave (at 73rd St), New York, NY 1...",40.778262,-73.978193,4a99ba54f964a520113020e3


In [10]:
# getting rating for each of the indian restaurants

i = 0
ny_dataframe_filtered['rating'] = 0.0
for venue_id in ny_dataframe_filtered['id']:
    print(venue_id)
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
        ny_dataframe_filtered.loc[i,'rating'] = result['response']['venue']['rating']
    except:
        print("no rating")
    
    i = i + 1
ny_dataframe_filtered

4a60aa67f964a520f9c01fe3
6.7
54c2bd96498eaf5142e3fe92
7.6
41575800f964a5202e1d1fe3
5.9
545eb9de498ed803ac5569c4
no rating
4e4e4d2fbd4101d0d7a73475
no rating
4aa573aaf964a5206d4820e3
6.2
4adb9df6f964a5206d2921e3
7.2
4b0dec08f964a520ae5223e3
7.5
42489a80f964a5208b201fe3
8.6
4a99ba54f964a520113020e3
5.9
4cf94bfa7f2db1f7796520d4
no rating
56ed855a498ef3bb022352c3
7.7


,name,categories,address,distance,formattedAddress,lat,lng,id,rating
0,Polash Indian restaurant,Indian Restaurant,2179 3rd Ave,2490,"[2179 3rd Ave (at 119th St.), New York, NY 100...",40.799745,-73.938747,4a60aa67f964a520f9c01fe3,6.7
1,Clove Indian Restaurant & Bar,Indian Restaurant,1592 Amsterdam Ave,2916,[1592 Amsterdam Ave (btwn 139th St & 138th St)...,40.821280,-73.950620,54c2bd96498eaf5142e3fe92,7.6
2,Ajanta Indian Restaurant,Indian Restaurant,1237 Amsterdam Ave,1475,"[1237 Amsterdam Ave, New York, NY 10027, Unite...",40.809905,-73.958824,41575800f964a5202e1d1fe3,5.9
3,SPICEHUT INDIAN RESTAURANT,Indian Restaurant,2036 2nd Ave,2449,"[2036 2nd Ave (105 Street), New York, NY 10029...",40.794198,-73.939837,545eb9de498ed803ac5569c4,0.0
4,Strength West Indian Restaurant,Indian Restaurant,429 Malcolm X Blvd,3077,"[429 Malcolm X Blvd, New York, NY 10037, Unite...",40.817880,-73.941921,4e4e4d2fbd4101d0d7a73475,0.0
5,Curry & Kebab,Indian Restaurant,931 Amsterdam Ave,263,"[931 Amsterdam Ave (W 106th St.), New York, NY...",40.800247,-73.965858,4aa573aaf964a5206d4820e3,6.2
6,Indian Tanpura,Indian Restaurant,564 Amsterdam Ave,1250,"[564 Amsterdam Ave (87th st), New York, NY 100...",40.788515,-73.974405,4adb9df6f964a5206d2921e3,7.2
7,Alachi Masala,Indian Restaurant,488 Amsterdam Ave,1559,"[488 Amsterdam Ave, New York, NY 10024, United...",40.786111,-73.976270,4b0dec08f964a520ae5223e3,7.5
8,Swagat,Indian Restaurant,411 Amsterdam Ave,1878,"[411 Amsterdam Ave (btwn 80th St & 79th St), N...",40.783573,-73.978030,42489a80f964a5208b201fe3,8.6
9,Indigo Indian Bistro,Indian Restaurant,283 Columbus Ave,2428,"[283 Columbus Ave (at 73rd St), New York, NY 1...",40.778262,-73.978193,4a99ba54f964a520113020e3,5.9


In [12]:
# adding 2 new columns "rating_result" and "dist" of type 'str'

ny_dataframe_filtered['rating_result'] = ny_dataframe_filtered['rating']
ny_dataframe_filtered['rating_result'] = ny_dataframe_filtered['rating_result'].astype(str)
ny_dataframe_filtered['rating_result'] = ny_dataframe_filtered['rating_result'].replace('0.0','not rated',regex=True)
ny_dataframe_filtered['dist'] = ny_dataframe_filtered['distance']
ny_dataframe_filtered['dist'] = ny_dataframe_filtered['dist'].astype(str)

ny_dataframe_filtered

,name,categories,address,distance,formattedAddress,lat,lng,id,rating,rating_result,dist
0,Polash Indian restaurant,Indian Restaurant,2179 3rd Ave,2490,"[2179 3rd Ave (at 119th St.), New York, NY 100...",40.799745,-73.938747,4a60aa67f964a520f9c01fe3,6.7,6.7,2490
1,Clove Indian Restaurant & Bar,Indian Restaurant,1592 Amsterdam Ave,2916,[1592 Amsterdam Ave (btwn 139th St & 138th St)...,40.821280,-73.950620,54c2bd96498eaf5142e3fe92,7.6,7.6,2916
2,Ajanta Indian Restaurant,Indian Restaurant,1237 Amsterdam Ave,1475,"[1237 Amsterdam Ave, New York, NY 10027, Unite...",40.809905,-73.958824,41575800f964a5202e1d1fe3,5.9,5.9,1475
3,SPICEHUT INDIAN RESTAURANT,Indian Restaurant,2036 2nd Ave,2449,"[2036 2nd Ave (105 Street), New York, NY 10029...",40.794198,-73.939837,545eb9de498ed803ac5569c4,0.0,not rated,2449
4,Strength West Indian Restaurant,Indian Restaurant,429 Malcolm X Blvd,3077,"[429 Malcolm X Blvd, New York, NY 10037, Unite...",40.817880,-73.941921,4e4e4d2fbd4101d0d7a73475,0.0,not rated,3077
5,Curry & Kebab,Indian Restaurant,931 Amsterdam Ave,263,"[931 Amsterdam Ave (W 106th St.), New York, NY...",40.800247,-73.965858,4aa573aaf964a5206d4820e3,6.2,6.2,263
6,Indian Tanpura,Indian Restaurant,564 Amsterdam Ave,1250,"[564 Amsterdam Ave (87th st), New York, NY 100...",40.788515,-73.974405,4adb9df6f964a5206d2921e3,7.2,7.2,1250
7,Alachi Masala,Indian Restaurant,488 Amsterdam Ave,1559,"[488 Amsterdam Ave, New York, NY 10024, United...",40.786111,-73.976270,4b0dec08f964a520ae5223e3,7.5,7.5,1559
8,Swagat,Indian Restaurant,411 Amsterdam Ave,1878,"[411 Amsterdam Ave (btwn 80th St & 79th St), N...",40.783573,-73.978030,42489a80f964a5208b201fe3,8.6,8.6,1878
9,Indigo Indian Bistro,Indian Restaurant,283 Columbus Ave,2428,"[283 Columbus Ave (at 73rd St), New York, NY 1...",40.778262,-73.978193,4a99ba54f964a520113020e3,5.9,5.9,2428


### Creating A Map With Marrakech Hotel and Nearby Indian Restaurants

In [13]:
#creating a map with the Marrakech Hotel and nearby Indian Restaurants

ny_restaurant_map = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=13) # generate map centred around Clarence Hotel

# add a green circle marker to represent 'Marrakech Hotel'
folium.features.CircleMarker(
    [ny_latitude, ny_longitude],
    radius=10,
    color='green',
    popup='Marrakech Hotel, 2688 Broadway, NY, USA',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(ny_restaurant_map)

# add the Indian restaurants near Marrakech Hotel in Manhattan, New York as blue circle markers
for lat, lng, label in zip(ny_dataframe_filtered.lat, ny_dataframe_filtered.lng, ny_dataframe_filtered.name+"-"+ny_dataframe_filtered.categories+" (rating: "+ny_dataframe_filtered.rating_result+" )"+" (distance: "+ny_dataframe_filtered.dist+"m)"):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(ny_restaurant_map)

ny_restaurant_map

### Putting a search query for nearby Art/History Museums at about 10 km radius frm the Marrakech Hotel

In [14]:
qry = 'Museum'
lat=ny_latitude
lng=ny_longitude
radius = 100000
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            qry)

results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5cbfec164c1f6743920b959c'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/museum_art_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d18f941735',
      'name': 'Art Museum',
      'pluralName': 'Art Museums',
      'primary': True,
      'shortName': 'Art Museum'}],
    'hasPerk': False,
    'id': '427c0500f964a52097211fe3',
    'location': {'address': '1000 5th Ave',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'crossStreet': 'btwn E 80th & E 84th St',
     'distance': 2155,
     'formattedAddress': ['1000 5th Ave (btwn E 80th & E 84th St)',
      'New York, NY 10028',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.77972902126812,
       'lng': -73.96341562271118}],
     'lat': 40.77972902126812,
     'lng': -73.96341562271118,
     'postalCode': '10028',
     'state': 'NY'},
    'name': 'The Me

###  Cleaning Of Json File And Insert Necessary Data Into Pandas Dataframe nearby_venues_filtered

In [15]:
venues = results['response']['venues']
nearby_venues = json_normalize(venues)
nearby_venues.head()

,categories,events.count,events.summary,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,NaN,NaN,False,427c0500f964a52097211fe3,1000 5th Ave,US,New York,United States,btwn E 80th & E 84th St,...,"[1000 5th Ave (btwn E 80th & E 84th St), New Y...","[{'lat': 40.77972902126812, 'lng': -73.9634156...",40.779729,-73.963416,NaN,10028,NY,The Metropolitan Museum of Art (Metropolitan M...,v-1556081686,NaN
1,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,NaN,NaN,False,4297b480f964a52062241fe3,200 Central Park W,US,New York,United States,btwn W 77th & W 81st St,...,"[200 Central Park W (btwn W 77th & W 81st St),...",NaN,40.781282,-73.973238,NaN,10024,NY,American Museum of Natural History,v-1556081686,NaN
2,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,NaN,NaN,False,4b65b072f964a520cff92ae3,200 Central Park W,US,New York,United States,btwn W 77th & W 81st St,...,"[200 Central Park W (btwn W 77th & W 81st St),...","[{'lat': 40.78097328556943, 'lng': -73.9730283...",40.780973,-73.973028,NaN,10024,NY,American Museum of Natural History Museum Shop,v-1556081686,NaN
3,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,NaN,NaN,False,4af0ab4cf964a52020de21e3,West 81st St,US,New York,United States,Central Park West,...,"[West 81st St (Central Park West), New York, N...","[{'lat': 40.7820428259257, 'lng': -73.97196681...",40.782043,-73.971967,NaN,10024,NY,MTA Subway - 81st St/Museum of Natural History...,v-1556081686,NaN
4,[{'icon': {'prefix': 'https://ss3.4sqi.net/img...,NaN,NaN,False,41706480f964a520a51d1fe3,1071 5th Ave,US,New York,United States,at E 89th St,...,"[1071 5th Ave (at E 89th St), New York, NY 101...","[{'lat': 40.78305146085593, 'lng': -73.9592652...",40.783051,-73.959265,NaN,10128,NY,Solomon R Guggenheim Museum,v-1556081686,32825424


In [16]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in nearby_venues.columns if col.startswith('location.')] + ['id']
nearby_venues_filtered = nearby_venues.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
nearby_venues_filtered['categories'] = nearby_venues_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
nearby_venues_filtered.columns = [column.split('.')[-1] for column in nearby_venues_filtered.columns]

nearby_venues_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Metropolitan Museum of Art (Metropolitan M...,Art Museum,1000 5th Ave,US,New York,United States,btwn E 80th & E 84th St,2155,"[1000 5th Ave (btwn E 80th & E 84th St), New Y...","[{'lat': 40.77972902126812, 'lng': -73.9634156...",40.779729,-73.963416,NaN,10028,NY,427c0500f964a52097211fe3
1,American Museum of Natural History,Science Museum,200 Central Park W,US,New York,United States,btwn W 77th & W 81st St,1988,"[200 Central Park W (btwn W 77th & W 81st St),...",NaN,40.781282,-73.973238,NaN,10024,NY,4297b480f964a52062241fe3
2,American Museum of Natural History Museum Shop,Souvenir Shop,200 Central Park W,US,New York,United States,btwn W 77th & W 81st St,2018,"[200 Central Park W (btwn W 77th & W 81st St),...","[{'lat': 40.78097328556943, 'lng': -73.9730283...",40.780973,-73.973028,NaN,10024,NY,4b65b072f964a520cff92ae3
3,MTA Subway - 81st St/Museum of Natural History...,Metro Station,West 81st St,US,New York,United States,Central Park West,1884,"[West 81st St (Central Park West), New York, N...","[{'lat': 40.7820428259257, 'lng': -73.97196681...",40.782043,-73.971967,NaN,10024,NY,4af0ab4cf964a52020de21e3
4,Solomon R Guggenheim Museum,Art Museum,1071 5th Ave,US,New York,United States,at E 89th St,1904,"[1071 5th Ave (at E 89th St), New York, NY 101...","[{'lat': 40.78305146085593, 'lng': -73.9592652...",40.783051,-73.959265,NaN,10128,NY,41706480f964a520a51d1fe3
5,Museum of Modern Art (MoMA),Art Museum,11 W 53rd St,US,New York,United States,btwn 5th & 6th Ave,4222,"[11 W 53rd St (btwn 5th & 6th Ave), New York, ...",NaN,40.761381,-73.976962,NaN,10019,NY,4af5a46af964a520b5fa21e3
6,Theodore Roosevelt Park (Museum Park),Park,W 77th St to W 81st St,US,New York,United States,Central Park West to Columbus Ave,1958,[W 77th St to W 81st St (Central Park West to ...,"[{'lat': 40.7818514, 'lng': -73.9747587, 'labe...",40.781851,-73.974759,NaN,10024,NY,4a80bb75f964a5200ff61fe3
7,Museum of the City of New York,History Museum,1220 5th Ave,US,New York,United States,at E 103rd St,1527,"[1220 5th Ave (at E 103rd St), New York, NY 10...","[{'lat': 40.792577607388715, 'lng': -73.952074...",40.792578,-73.952075,NaN,10029,NY,4530db98f964a520623b1fe3
8,Whitney Museum of American Art,Art Museum,99 Gansevoort St,US,New York,United States,btwn 10th Ave & Washington St,7429,[99 Gansevoort St (btwn 10th Ave & Washington ...,"[{'lat': 40.739505596373306, 'lng': -74.008860...",40.739506,-74.008861,Meatpac,10014,NY,421a7600f964a5209d1f1fe3
9,Cooper Hewitt Smithsonian Design Museum,Art Museum,2 E 91st St,US,New York,United States,at 5th Ave,1832,"[2 E 91st St (at 5th Ave), New York, NY 10128,...","[{'lat': 40.78433310928044, 'lng': -73.9577650...",40.784333,-73.957765,NaN,10128,NY,4a2fc4d3f964a520da981fe3


In [17]:
# dropping unnecessary columns and then resetting index

nearby_venues_filtered.drop(nearby_venues_filtered.columns[[3,4,5,6,9,12,13,14]], axis=1, inplace=True)
nearby_venues_filtered = nearby_venues_filtered[(nearby_venues_filtered.categories == 'Art Museum') | (nearby_venues_filtered.categories == 'History Museum')]
nearby_venues_filtered = nearby_venues_filtered.dropna(subset=['address'])
nearby_venues_filtered = nearby_venues_filtered.reset_index()
del nearby_venues_filtered['index']
nearby_venues_filtered

,name,categories,address,distance,formattedAddress,lat,lng,id
0,The Metropolitan Museum of Art (Metropolitan M...,Art Museum,1000 5th Ave,2155,"[1000 5th Ave (btwn E 80th & E 84th St), New Y...",40.779729,-73.963416,427c0500f964a52097211fe3
1,Solomon R Guggenheim Museum,Art Museum,1071 5th Ave,1904,"[1071 5th Ave (at E 89th St), New York, NY 101...",40.783051,-73.959265,41706480f964a520a51d1fe3
2,Museum of Modern Art (MoMA),Art Museum,11 W 53rd St,4222,"[11 W 53rd St (btwn 5th & 6th Ave), New York, ...",40.761381,-73.976962,4af5a46af964a520b5fa21e3
3,Museum of the City of New York,History Museum,1220 5th Ave,1527,"[1220 5th Ave (at E 103rd St), New York, NY 10...",40.792578,-73.952075,4530db98f964a520623b1fe3
4,Whitney Museum of American Art,Art Museum,99 Gansevoort St,7429,[99 Gansevoort St (btwn 10th Ave & Washington ...,40.739506,-74.008861,421a7600f964a5209d1f1fe3
5,Cooper Hewitt Smithsonian Design Museum,Art Museum,2 E 91st St,1832,"[2 E 91st St (at 5th Ave), New York, NY 10128,...",40.784333,-73.957765,4a2fc4d3f964a520da981fe3
6,Brooklyn Museum,Art Museum,200 Eastern Pkwy,14167,"[200 Eastern Pkwy (at Washington Ave), Brookly...",40.671521,-73.963677,427d5680f964a520a8211fe3
7,Ellis Island Immigration Museum,History Museum,Ellis Island,12625,"[Ellis Island, Jersey City, NJ 07305, United S...",40.699028,-74.039599,4a44cd95f964a5208fa71fe3
8,New-York Historical Society Museum & Library,History Museum,170 Central Park W,2218,"[170 Central Park W (at W 77th St), New York, ...",40.779271,-73.973905,4b48d37df964a520d55826e3
9,Museum for African Art,Art Museum,1280 5th Ave,1682,"[1280 5th Ave (110th and 5th), New York, NY 10...",40.796099,-73.948619,4cacecee2776b60c46be550d


In [18]:
# retrieving rating for each of the museums searched

i = 0
nearby_venues_filtered['rating'] = 0.0
for venue_id in nearby_venues_filtered['id']:
    print(venue_id)
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    try:
        print(result['response']['venue']['rating'])
        nearby_venues_filtered.loc[i,'rating'] = result['response']['venue']['rating']
    except:
        print("no rating")
    
    i = i + 1
nearby_venues_filtered

427c0500f964a52097211fe3
9.7
41706480f964a520a51d1fe3
9.0
4af5a46af964a520b5fa21e3
9.5
4530db98f964a520623b1fe3
9.1
421a7600f964a5209d1f1fe3
9.4
4a2fc4d3f964a520da981fe3
9.3
427d5680f964a520a8211fe3
9.5
4a44cd95f964a5208fa71fe3
8.9
4b48d37df964a520d55826e3
8.9
4cacecee2776b60c46be550d
7.4
49f5135cf964a5208e6b1fe3
9.2
4a6cbb32f964a52076d11fe3
8.6
4bc8ceda0687ef3bd7a3d8cc
6.8
4ab2c893f964a520566c20e3
9.3
47b6cfccf964a520af4d1fe3
9.2
4b6da13ff964a520b3812ce3
7.3
4750accef964a520b24c1fe3
8.5
4be0a0564f15c92801ddcb0b
8.8
4a917aa4f964a520541a20e3
9.3
4a82cffff964a52065f91fe3
8.9
4bac00a1f964a52025dd3ae3
9.0
45727340f964a520733e1fe3
9.3
49d4bb1bf964a5204b5c1fe3
8.4


,name,categories,address,distance,formattedAddress,lat,lng,id,rating
0,The Metropolitan Museum of Art (Metropolitan M...,Art Museum,1000 5th Ave,2155,"[1000 5th Ave (btwn E 80th & E 84th St), New Y...",40.779729,-73.963416,427c0500f964a52097211fe3,9.7
1,Solomon R Guggenheim Museum,Art Museum,1071 5th Ave,1904,"[1071 5th Ave (at E 89th St), New York, NY 101...",40.783051,-73.959265,41706480f964a520a51d1fe3,9.0
2,Museum of Modern Art (MoMA),Art Museum,11 W 53rd St,4222,"[11 W 53rd St (btwn 5th & 6th Ave), New York, ...",40.761381,-73.976962,4af5a46af964a520b5fa21e3,9.5
3,Museum of the City of New York,History Museum,1220 5th Ave,1527,"[1220 5th Ave (at E 103rd St), New York, NY 10...",40.792578,-73.952075,4530db98f964a520623b1fe3,9.1
4,Whitney Museum of American Art,Art Museum,99 Gansevoort St,7429,[99 Gansevoort St (btwn 10th Ave & Washington ...,40.739506,-74.008861,421a7600f964a5209d1f1fe3,9.4
5,Cooper Hewitt Smithsonian Design Museum,Art Museum,2 E 91st St,1832,"[2 E 91st St (at 5th Ave), New York, NY 10128,...",40.784333,-73.957765,4a2fc4d3f964a520da981fe3,9.3
6,Brooklyn Museum,Art Museum,200 Eastern Pkwy,14167,"[200 Eastern Pkwy (at Washington Ave), Brookly...",40.671521,-73.963677,427d5680f964a520a8211fe3,9.5
7,Ellis Island Immigration Museum,History Museum,Ellis Island,12625,"[Ellis Island, Jersey City, NJ 07305, United S...",40.699028,-74.039599,4a44cd95f964a5208fa71fe3,8.9
8,New-York Historical Society Museum & Library,History Museum,170 Central Park W,2218,"[170 Central Park W (at W 77th St), New York, ...",40.779271,-73.973905,4b48d37df964a520d55826e3,8.9
9,Museum for African Art,Art Museum,1280 5th Ave,1682,"[1280 5th Ave (110th and 5th), New York, NY 10...",40.796099,-73.948619,4cacecee2776b60c46be550d,7.4


In [19]:
# adding 2 new columns "rating_result" and "dist" of type str into nearby_venues_filtered

nearby_venues_filtered['rating_result'] = nearby_venues_filtered['rating']
nearby_venues_filtered['rating_result'] = nearby_venues_filtered['rating_result'].astype(str)
nearby_venues_filtered['rating_result'] = nearby_venues_filtered['rating_result'].replace('0.0','not rated',regex=True)
nearby_venues_filtered['dist'] = nearby_venues_filtered['distance']
nearby_venues_filtered['dist'] = nearby_venues_filtered['dist'].astype(str)
nearby_venues_filtered

,name,categories,address,distance,formattedAddress,lat,lng,id,rating,rating_result,dist
0,The Metropolitan Museum of Art (Metropolitan M...,Art Museum,1000 5th Ave,2155,"[1000 5th Ave (btwn E 80th & E 84th St), New Y...",40.779729,-73.963416,427c0500f964a52097211fe3,9.7,9.7,2155
1,Solomon R Guggenheim Museum,Art Museum,1071 5th Ave,1904,"[1071 5th Ave (at E 89th St), New York, NY 101...",40.783051,-73.959265,41706480f964a520a51d1fe3,9.0,9.0,1904
2,Museum of Modern Art (MoMA),Art Museum,11 W 53rd St,4222,"[11 W 53rd St (btwn 5th & 6th Ave), New York, ...",40.761381,-73.976962,4af5a46af964a520b5fa21e3,9.5,9.5,4222
3,Museum of the City of New York,History Museum,1220 5th Ave,1527,"[1220 5th Ave (at E 103rd St), New York, NY 10...",40.792578,-73.952075,4530db98f964a520623b1fe3,9.1,9.1,1527
4,Whitney Museum of American Art,Art Museum,99 Gansevoort St,7429,[99 Gansevoort St (btwn 10th Ave & Washington ...,40.739506,-74.008861,421a7600f964a5209d1f1fe3,9.4,9.4,7429
5,Cooper Hewitt Smithsonian Design Museum,Art Museum,2 E 91st St,1832,"[2 E 91st St (at 5th Ave), New York, NY 10128,...",40.784333,-73.957765,4a2fc4d3f964a520da981fe3,9.3,9.3,1832
6,Brooklyn Museum,Art Museum,200 Eastern Pkwy,14167,"[200 Eastern Pkwy (at Washington Ave), Brookly...",40.671521,-73.963677,427d5680f964a520a8211fe3,9.5,9.5,14167
7,Ellis Island Immigration Museum,History Museum,Ellis Island,12625,"[Ellis Island, Jersey City, NJ 07305, United S...",40.699028,-74.039599,4a44cd95f964a5208fa71fe3,8.9,8.9,12625
8,New-York Historical Society Museum & Library,History Museum,170 Central Park W,2218,"[170 Central Park W (at W 77th St), New York, ...",40.779271,-73.973905,4b48d37df964a520d55826e3,8.9,8.9,2218
9,Museum for African Art,Art Museum,1280 5th Ave,1682,"[1280 5th Ave (110th and 5th), New York, NY 10...",40.796099,-73.948619,4cacecee2776b60c46be550d,7.4,7.4,1682


### Generating a map of Marrakech Hotel with surrounding Art/History Museums

In [20]:
# Generating a map of Marrakech Hotel with surrounding Art/History Museums

nearby_venues_map = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=13) # generate map centred around Clarence Hotel

# add a green circle marker to represent 'Marakech Hotel'
folium.features.CircleMarker(
    [ny_latitude, ny_longitude],
    radius=10,
    color='green',
    popup='Marrakech Hotel, 2688 Broadway, NY, USA',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(nearby_venues_map)

# add the Art/History Museums near Marrakech Hotel in Manhattan as red circle markers
for lat, lng, label in zip(nearby_venues_filtered.lat, nearby_venues_filtered.lng, nearby_venues_filtered.name+" (rating:"+nearby_venues_filtered.rating_result+")"+" (distance: "+nearby_venues_filtered.dist+"m)"):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(nearby_venues_map)

nearby_venues_map

### Generating a Map with The Marrakech Hotel with nearby Indian Restaurants and nearby Art/History Museums
#### The pop up gives distance from Marrakech hotel as well as the rating

In [21]:
nearby_map = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=13) # generate map centred around Marrakech Hotel

# add a green circle marker to represent 'Marakech Hotel'
folium.features.CircleMarker(
    [ny_latitude, ny_longitude],
    radius=10,
    color='green',
    popup='Marrakech Hotel, 2688 Broadway, NY, USA',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(nearby_map)

# add the nearby Art/History Museums near Marrakech Hotelin Manhattan, New York as big red circle markers
for lat, lng, label in zip(nearby_venues_filtered.lat, nearby_venues_filtered.lng, nearby_venues_filtered.name+" (rating:"+nearby_venues_filtered.rating_result+")"+" (distance: "+nearby_venues_filtered.dist+"m)"):
    folium.features.CircleMarker(
        [lat, lng],
        radius=7,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(nearby_map)

# add the Indian restaurants near Marrakech Hotel in Manhattan, New York as smaller blue circle markers
for lat, lng, label in zip(ny_dataframe_filtered.lat, ny_dataframe_filtered.lng, ny_dataframe_filtered.name+"-"+ny_dataframe_filtered.categories+" (rating: "+ny_dataframe_filtered.rating_result+" )"+" (distance: "+ny_dataframe_filtered.dist+"m)"):
    folium.features.CircleMarker(
        [lat, lng],
        radius=3,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(nearby_map)

nearby_map

## Clustering the Restaurants And Museums On The Basis Of Radial Distances From The Marrakech Hotel

In [22]:
# Run k-means to cluster the restaurants into 5 clusters based on radial distance from Marrakech hotel

kclusters=5
res_grouped_clustering = ny_dataframe_filtered.drop('categories', 1)
res_grouped_clustering = res_grouped_clustering.drop('address', 1)
res_grouped_clustering = res_grouped_clustering.drop('dist', 1)
res_grouped_clustering = res_grouped_clustering.drop('formattedAddress', 1)
res_grouped_clustering = res_grouped_clustering.drop('id', 1)
res_grouped_clustering = res_grouped_clustering.drop('rating_result', 1)
res_grouped_clustering = res_grouped_clustering.drop('name', 1)

display(res_grouped_clustering)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(res_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

,distance,lat,lng,rating
0,2490,40.799745,-73.938747,6.7
1,2916,40.821280,-73.950620,7.6
2,1475,40.809905,-73.958824,5.9
3,2449,40.794198,-73.939837,0.0
4,3077,40.817880,-73.941921,0.0
5,263,40.800247,-73.965858,6.2
6,1250,40.788515,-73.974405,7.2
7,1559,40.786111,-73.976270,7.5
8,1878,40.783573,-73.978030,8.6
9,2428,40.778262,-73.978193,5.9


array([1, 3, 2, 1, 3, 0, 2, 2, 4, 1, 1, 1], dtype=int32)

In [23]:
kmeans.cluster_centers_.tolist()

[[263.0, 40.8002471075292, -73.96585761960743, 6.2],
 [2479.8, 40.78932571465983, -73.94930576286436, 4.0600000000000005],
 [1428.0, 40.79484374298929, -73.96983288931166, 6.866666666666667],
 [2996.5, 40.81958000866495, -73.94627030832424, 3.8],
 [1878.0, 40.783573082941615, -73.97803042905113, 8.6]]

In [24]:
ny_dataframe_filtered['Cluster Labels'] = kmeans.labels_
display(ny_dataframe_filtered)

,name,categories,address,distance,formattedAddress,lat,lng,id,rating,rating_result,dist,Cluster Labels
0,Polash Indian restaurant,Indian Restaurant,2179 3rd Ave,2490,"[2179 3rd Ave (at 119th St.), New York, NY 100...",40.799745,-73.938747,4a60aa67f964a520f9c01fe3,6.7,6.7,2490,1
1,Clove Indian Restaurant & Bar,Indian Restaurant,1592 Amsterdam Ave,2916,[1592 Amsterdam Ave (btwn 139th St & 138th St)...,40.821280,-73.950620,54c2bd96498eaf5142e3fe92,7.6,7.6,2916,3
2,Ajanta Indian Restaurant,Indian Restaurant,1237 Amsterdam Ave,1475,"[1237 Amsterdam Ave, New York, NY 10027, Unite...",40.809905,-73.958824,41575800f964a5202e1d1fe3,5.9,5.9,1475,2
3,SPICEHUT INDIAN RESTAURANT,Indian Restaurant,2036 2nd Ave,2449,"[2036 2nd Ave (105 Street), New York, NY 10029...",40.794198,-73.939837,545eb9de498ed803ac5569c4,0.0,not rated,2449,1
4,Strength West Indian Restaurant,Indian Restaurant,429 Malcolm X Blvd,3077,"[429 Malcolm X Blvd, New York, NY 10037, Unite...",40.817880,-73.941921,4e4e4d2fbd4101d0d7a73475,0.0,not rated,3077,3
5,Curry & Kebab,Indian Restaurant,931 Amsterdam Ave,263,"[931 Amsterdam Ave (W 106th St.), New York, NY...",40.800247,-73.965858,4aa573aaf964a5206d4820e3,6.2,6.2,263,0
6,Indian Tanpura,Indian Restaurant,564 Amsterdam Ave,1250,"[564 Amsterdam Ave (87th st), New York, NY 100...",40.788515,-73.974405,4adb9df6f964a5206d2921e3,7.2,7.2,1250,2
7,Alachi Masala,Indian Restaurant,488 Amsterdam Ave,1559,"[488 Amsterdam Ave, New York, NY 10024, United...",40.786111,-73.976270,4b0dec08f964a520ae5223e3,7.5,7.5,1559,2
8,Swagat,Indian Restaurant,411 Amsterdam Ave,1878,"[411 Amsterdam Ave (btwn 80th St & 79th St), N...",40.783573,-73.978030,42489a80f964a5208b201fe3,8.6,8.6,1878,4
9,Indigo Indian Bistro,Indian Restaurant,283 Columbus Ave,2428,"[283 Columbus Ave (at 73rd St), New York, NY 1...",40.778262,-73.978193,4a99ba54f964a520113020e3,5.9,5.9,2428,1


In [25]:
# Run k-means to cluster the art/history museums into 5 clusters based on radial distance from Marrakech Hotel

kclusters=5
venues_grouped_clustering = nearby_venues_filtered.drop('categories', 1)
venues_grouped_clustering = venues_grouped_clustering.drop('address', 1)
venues_grouped_clustering = venues_grouped_clustering.drop('dist', 1)
venues_grouped_clustering = venues_grouped_clustering.drop('formattedAddress', 1)
venues_grouped_clustering = venues_grouped_clustering.drop('id', 1)
venues_grouped_clustering = venues_grouped_clustering.drop('rating_result', 1)
venues_grouped_clustering = venues_grouped_clustering.drop('name', 1)

display(venues_grouped_clustering)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

,distance,lat,lng,rating
0,2155,40.779729,-73.963416,9.7
1,1904,40.783051,-73.959265,9.0
2,4222,40.761381,-73.976962,9.5
3,1527,40.792578,-73.952075,9.1
4,7429,40.739506,-74.008861,9.4
5,1832,40.784333,-73.957765,9.3
6,14167,40.671521,-73.963677,9.5
7,12625,40.699028,-74.039599,8.9
8,2218,40.779271,-73.973905,8.9
9,1682,40.796099,-73.948619,7.4


array([0, 0, 3, 0, 4, 0, 2, 2, 0, 0, 3, 3, 0, 1, 4, 0, 1, 3, 3, 0, 1, 2, 2], dtype=int32)

In [26]:
kmeans.cluster_centers_.tolist()

[[2036.4444444444443, 40.78672177108284, -73.9603476160756, 8.488888888888889],
 [9009.666666666666,
  40.720243776642796,
  -73.99407183261413,
  8.933333333333334],
 [12533.25, 40.69173481165694, -74.00280879077096, 9.025],
 [4364.0, 40.77795277861192, -73.96463918038903, 9.08],
 [7207.0, 40.739807760281224, -74.00333392185533, 9.3]]

In [27]:
nearby_venues_filtered['Cluster Labels'] = kmeans.labels_
display(nearby_venues_filtered)

,name,categories,address,distance,formattedAddress,lat,lng,id,rating,rating_result,dist,Cluster Labels
0,The Metropolitan Museum of Art (Metropolitan M...,Art Museum,1000 5th Ave,2155,"[1000 5th Ave (btwn E 80th & E 84th St), New Y...",40.779729,-73.963416,427c0500f964a52097211fe3,9.7,9.7,2155,0
1,Solomon R Guggenheim Museum,Art Museum,1071 5th Ave,1904,"[1071 5th Ave (at E 89th St), New York, NY 101...",40.783051,-73.959265,41706480f964a520a51d1fe3,9.0,9.0,1904,0
2,Museum of Modern Art (MoMA),Art Museum,11 W 53rd St,4222,"[11 W 53rd St (btwn 5th & 6th Ave), New York, ...",40.761381,-73.976962,4af5a46af964a520b5fa21e3,9.5,9.5,4222,3
3,Museum of the City of New York,History Museum,1220 5th Ave,1527,"[1220 5th Ave (at E 103rd St), New York, NY 10...",40.792578,-73.952075,4530db98f964a520623b1fe3,9.1,9.1,1527,0
4,Whitney Museum of American Art,Art Museum,99 Gansevoort St,7429,[99 Gansevoort St (btwn 10th Ave & Washington ...,40.739506,-74.008861,421a7600f964a5209d1f1fe3,9.4,9.4,7429,4
5,Cooper Hewitt Smithsonian Design Museum,Art Museum,2 E 91st St,1832,"[2 E 91st St (at 5th Ave), New York, NY 10128,...",40.784333,-73.957765,4a2fc4d3f964a520da981fe3,9.3,9.3,1832,0
6,Brooklyn Museum,Art Museum,200 Eastern Pkwy,14167,"[200 Eastern Pkwy (at Washington Ave), Brookly...",40.671521,-73.963677,427d5680f964a520a8211fe3,9.5,9.5,14167,2
7,Ellis Island Immigration Museum,History Museum,Ellis Island,12625,"[Ellis Island, Jersey City, NJ 07305, United S...",40.699028,-74.039599,4a44cd95f964a5208fa71fe3,8.9,8.9,12625,2
8,New-York Historical Society Museum & Library,History Museum,170 Central Park W,2218,"[170 Central Park W (at W 77th St), New York, ...",40.779271,-73.973905,4b48d37df964a520d55826e3,8.9,8.9,2218,0
9,Museum for African Art,Art Museum,1280 5th Ave,1682,"[1280 5th Ave (110th and 5th), New York, NY 10...",40.796099,-73.948619,4cacecee2776b60c46be550d,7.4,7.4,1682,0


# EXAMINE THE CLUSTERS

## Generating a Clustered Map on Basis of Radial Distance with The Marrakech Hotel with nearby Indian Restaurants and nearby Art/History Museums

### The small cricles denote the Indian Restaurants
### The bigger cirlce denotes the Art/History Museums
### The largest green circle denotes the Marrakech Hotel

#### The pop up gives distance from Marrakech hotel as well as the rating¶

In [28]:
# create a map for the location of Marrakech Hotel in New York
map_clusters = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=12)


# add a Large green circle marker to represent 'Marakech Hotel'
folium.features.CircleMarker(
    [ny_latitude, ny_longitude],
    radius=14,
    color='green',
    popup='Marrakech Hotel, 2688 Broadway, NY, USA',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(map_clusters)


# set the colors for the 5 different clusters for Indian restaurants

rainbow_1=['red','yellow','brown','blue','orange']

# add the Indian restaurants near Marrakech Hotel in Manhattan, New York as small circle markers of radius 3

for lat, lon, poi, cluster, cat, dis, rtg in zip(ny_dataframe_filtered['lat'], ny_dataframe_filtered['lng'], ny_dataframe_filtered['name'], ny_dataframe_filtered['Cluster Labels'], ny_dataframe_filtered.categories,ny_dataframe_filtered.dist,ny_dataframe_filtered.rating_result):
    label = folium.Popup(str(poi)+" (Category: " + cat +")"+ "(distance: " + dis +"m)"+ "(rating: " + rtg + ")"+ " Cluster " + str(cluster), parse_html=True)
    folium.features.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow_1[cluster-1],
        fill=True,
        fill_color=rainbow_1[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# set the colors for the 5 different clusters for Art/History Museums

rainbow_2=['magenta','cyan','darkorchid','dimgray','lightseagreen']

    
# add the Art/History Museums near Marrakech Hotel in Manhattan, New York as big circle markers of radius 7

for lat, lon, poi, cluster,cat,dis,rtg in zip(nearby_venues_filtered['lat'], nearby_venues_filtered['lng'], nearby_venues_filtered['name'], nearby_venues_filtered['Cluster Labels'], nearby_venues_filtered.categories, nearby_venues_filtered.dist,nearby_venues_filtered.rating_result):
    label = folium.Popup(str(poi)+" (Category: " + cat +")"+ "(distance: " + dis +"m)"+ "(rating: " + rtg + ")"+ " Cluster " + str(cluster), parse_html=True)
    folium.features.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow_2[cluster-1],
        fill=True,
        fill_color=rainbow_2[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters